In [2]:
import pandas as pd
import numpy as np
import csv
import re

#Deberia revisar lo de Counter que lo uso bastante y no conoxco muy bien la libreria
from collections import Counter


In [357]:
def texto_csv(df, column):
    
    #name of the document: as column_name_column.csv
    path = f'column_{column}.csv'
    
    """ Hay que quitar los vacios primero - lo he sacado de la funcion
    por no tranformar la tabla aqui """
   
    #2. We write every colum as a new line in the .csv
    non_empty_values = [str(i) for i in df[column]]
    
    with open(path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows([[value] for value in non_empty_values]) 
        
    print(f'Column successfully exported to {path}')

In [44]:
df = pd.read_csv("../data/srape-mercadona_2024-03-08.csv")

# 1. EXPLORE

In [81]:
df.sample()

#categoria, subcategoria y descripcion - las dejamos como están. 

,categoria,subcategoria,descripcion,unidades,precio
1039,Bodega,Licores,Pacharán Oibar,Botella 1 L,"6,90 € /ud."


In [46]:
def dupli_rows(df):
    "How many duplicate rows there is"
    duplicates = df.shape[0] - df.drop_duplicates().shape[0]
    print(f'There is {duplicates} duplicate rows')
    return 

dupli_rows(df)

There is 19 duplicate rows


In [47]:
df.drop_duplicates(inplace=True)

In [48]:
# All collumns have values. 
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5016 entries, 0 to 5034
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   categoria     5016 non-null   object
 1   subcategoria  5016 non-null   object
 2   descripcion   5016 non-null   object
 3   unidades      5016 non-null   object
 4   precio        5016 non-null   object
dtypes: object(5)
memory usage: 235.1+ KB


# 2. TRANFORM

##  - PRECIO

- No hay valores nulos

- El precio puede contener o 2 informaciones o 3:
    - El precio y a que hace referencia ese precio (pack)
    - El precio, precio si esta descontado y a que hace referencia ese precio(pack)

- Creamos una funcion que aplicamos a todos los elementos de la columna precio para formaterla. Aqui extraeremos los tres valores comentados antes. Cuando no haya descuento --> np.nan
    - price_info --> string
    - price_eu and price_disc_eu --> float
    
  **Note:** Interesante aqui el tema de apply(function) y luego el lambda. 
  
- Eliminamos la columna original

**NOTE:** En price_info, los valores son bastante consistentes siendo la mayoria de valoes (ud/pack). No obstante hay 14 valores que son de tipo ( xxx g) ej 500 g. Veremos a ver que hacemos. si los descartamos o como los tranformamos. No son una parte muy relevante del conjunto de datos ( 14 de 5035) pero tener en cuenta que hay que revisarlo: 


**Tranform Price column**

In [261]:
def format_price(x):
    
    """
    Decompose and fromate values for the colum price
    
    input: string 
    output: lsit, values of each key already casted:
        info: pos[0]: str 
        valores: pos[1]: float
        descuento: pos[2]: float/np.nan(float) (where there is a discount or not)
    
    """
    
    # 1. Replace the € simbol and splits by the "/" ch -- generates a list of 2 elements (nums - text)
    x = x.replace("€", "").split("/")
    
    # 2. Split by the "/" to extract the info, keep last part(always at the end)
    info = x[-1].replace(".", "") #(str)
    
    # 3. Splits by the "/" to extract the num values:
    num_values = x[0:-1][0].replace("," , ".").strip().split(" ")
    
    # 4. Extracts and formats price value:
    price = float(num_values[0])
    
    # 5. Extracts and formats discount_price if not dicount np.nan
    discount = [np.nan if len(num_values) == 1 else float(num_values[1])][0]
    
    return [info, price, discount]

In [277]:
# Apply the function and get the element for each colum: 
df["price_info"] = df["precio"].apply(format_price).apply(lambda x:x[0])
df["price_eu"] = df["precio"].apply(format_price).apply(lambda x:x[1])
df["price_disc_eu"] = df["precio"].apply(format_price).apply(lambda x:x[2])

#Drop old column:
df.drop("precio", axis=1, inplace=True)

df.sample()

## - MARCA

Hay tres marcas principales que se van a repetir en casi la mitad de articulos, que son los nombres
de las marcas propias de mercadona: 


In [153]:
"""Funcion que te devuelve la marca si el texto la contiene"""
def marca(x):
    marcas = ["Hacendado", "Deliplus", "Bosque Verde", "Coca-Cola"]
    
    """otras_marcas = ["Hacendado", "Deliplus", "Bosque Verde", "Coca-Cola","Compy", "L'Oréal", "Delikuit", 
                "Maybelline", "Gillette", "Nivea", "Puleva", "Incarlopsa", "Tampax", "Gallina Blanca", 
                "El Pozo", "Hero baby", "Finish", "Garnier", "Rimmel London", "Pantene", "Evax", 
                "Dolce Gusto", "Entrepinares", "Mahou"] """
    
    
    return [i for i in marcas if i in x]

In [154]:
#Aplicamos la funcion, como realmente devuelve una lista, si esta vacia (no match) --> np.nan
df["marca"] = df["descripcion"].apply(marca).apply(lambda x: x[0] if x != [] else np.nan)

In [155]:
#Quiero ver cuantas filas no hemos identificado la marca 
df['marca'].isna().sum()

1984

Ver si queremos sacar el resto de marcas o no. Tener en cuenta que todo lo de la carne y tal creo que no 
pone que es acendado pero si que lo es, ver como vamos a hacer eso por si queremos comparar marcas blancas solo

Para intentar afinar un poco mas y sacar mas marcas, vamos a intentar lo siguiente: 

Me he dado cuenta de que las marcas estan siempre escritas La primera en  mayuscula.
La marca y la primera palabra de la descripcion, que suele hacer referencia al tipo de articulo
Nunca ponen la Marca lo primero, por lo que si esta en mayudculas sera o el primer elemento 
de la descripcion(hace referencia al producto) o la marca. 

In [117]:
#Dividimos las descripciones por palbras
lista = df["descripcion"].unique()
words_list = [ i.strip().split(" ") for i in lista]

In [124]:
# eliminamos la primera palabra de cada descirpcion( En mayusculas pero no es una marca)
palabras = []

for i in words_list:
    a = i[1:]
    for w in a:
        palabras.append(w)
        
#Nos queda una lista de palabras donde la mayoria de palabras estaran en minusculas, 
#sin embargo sabemos que las marcas tienen la primera letra en mayuscula: 

In [129]:
pattern = r"(\b[A-Z][a-zA-Z]*\b)" #Primera letra en mayucula
#Quitamos TODAS las palabras que no cumplan con el patron
clave = [word for word in palabras if re.match(pattern, word)]

In [150]:
"""Aqui podemos ver toas las palabras que empiezan por mayusculas y como se repiten, ya veremos
cuanto queremo pulir esto"""

#Counter(clave)

'Aqui podemos ver toas las palabras que empiezan por mayusculas y como se repiten, ya veremos\ncuanto queremo pulir esto'

In [146]:
# Esta es la lista de otras marcas que podemos incluir en la funcion: 

otras_marcas = ["Hacendado", "Deliplus", "Bosque Verde", "Coca-Cola","Compy", "L'Oréal", "Delikuit", 
                "Maybelline", "Gillette", "Nivea", "Puleva", "Incarlopsa", "Tampax", "Gallina Blanca", 
                "El Pozo", "Hero baby", "Finish", "Garnier", "Rimmel London", "Pantene", "Evax", 
                "Dolce Gusto", "Entrepinares"]

### UNIDADES:

In [156]:
df["unidades"] = df["unidades"].str.replace("aprox.", "")

In [163]:
df.sample()

,categoria,subcategoria,descripcion,unidades,precio,marca,cantidad
622,"Azúcar, caramelos y chocolate",Azúcar y edulcorante,Edulcorante en sobres sacarina Hacendado,Caja 60 sobres (60 g),"1,15 € /ud.",Hacendado,60 g


In [159]:
pattern = r'((?:\d+,)?\d+\s*(?:g|[Kk]g|L|ml))'

In [160]:
df["cantidad"] =  df["unidades"].str.extract(pattern)

In [164]:
df.sample()

,categoria,subcategoria,descripcion,unidades,precio,marca,cantidad
951,Bodega,Cerveza,Cerveza especial mini Mahou,Botellín 200 ml,"0,59 € /ud.",NaN,200 ml


In [288]:
pattern = r"(\b[A-Z][a-zA-Z]*\b)" #Primera letra en mayucula

1396

In [358]:
texto_csv(f_df, "unidades")

Column successfully exported to column_unidades.csv
